In [4]:
import csv
import requests
import os
import sys
import warnings
from datetime import datetime, timedelta
from sqlalchemy.sql import func

caminho_commons = os.path.join('..', '..', 'ajna_docs', 'commons')
caminho_virasana = os.path.join('..', '..', 'ajna_docs', 'virasana')
sys.path.append('..')
sys.path.append(caminho_commons)
sys.path.append(caminho_virasana)

from ajna_commons.utils.sanitiza import sanitizar, mongo_sanitizar
from bhadrasana.models.ovr import OVR
from bhadrasana.models.ovr import Recinto
from bhadrasana.models import db_session
warnings.simplefilter('ignore')

In [5]:
            #'https://jupapi.org.br/api/sepes/Pesagem/token
DTE_TOKEN = 'https://www.janelaunicaportuaria.org.br/ws_homologacao/sepes/api/Pesagem/token' 
    #''https://jupapi.org.br/api/sepes/Pesagem/token'
DTE_URL = 'https://www.janelaunicaportuaria.org.br/ws_homologacao/sepes/api/ConsultaFMA'
with open(os.path.join(caminho_virasana, 'virasana', 'integracao', 'dte.info')) as dte_info:
    linha = dte_info.readline()
DTE_USERNAME = linha.split(',')[0]
DTE_PASSWORD = linha.split(',')[1]

In [6]:
with open(os.path.join(caminho_virasana, 'virasana', 'integracao', 'recintos.csv'), encoding='utf-8') as csv_in:
    reader = csv.reader(csv_in)
    recintos_list = [row for row in reader]

def get_token_dte(username=DTE_USERNAME, password=DTE_PASSWORD):
    data = {'username': username, 'password': password, 'grant_type': 'password'}
    # print(data)
    r = requests.post(DTE_TOKEN, data=data, verify=False)
    try:
        token = r.json().get('access_token')
    except Exception as err:
        print(r.url, r.status_code, r.text)
        raise(err)
    return token

token = get_token_dte()
token

'4tB0ravhQr6Xk3lrXM3dcpGlNtv5yFC1b37Sfwjuq0P4RHxe0cZhnWvZndvKN5abhZqQM6j1PEFW8GlMDf49m0HLPSUU2xopTpZaifPTty5jmLyKTslNLgWKG9YHglEii3Y1SkbMc4sUBUDZGXOIHF9F1p2qLNoFN749K44n2gOrONnz2JhJQdTjFxgs-AYcauSH3qZrqITeFRey_ujuEy16tsIzv2vdIdNTLDJAZIszLu2WNWejmeguZcyfGv60ofgjaEAFyxs_V8SfSShkDPEMwYhfsuwzK3fAjllkYCg'

In [9]:
def get_lista_fma(cod_recinto, start, end):
    token = get_token_dte()
    payload = {'data_inicio': datetime.strftime(start, '%Y-%m-%d'),
           'data_fim': datetime.strftime(end, '%Y-%m-%d'),
           'cod_recinto': cod_recinto}
    headers = {'Authorization': 'Bearer ' + token}
    r = requests.get(DTE_URL, headers=headers, params=payload)
    if r.status_code != 200:
        raise Exception(r.text)
    return r.json()['JUP_WS']['FMA_Eletronica']['Lista_FMA']
                  
start = datetime.strptime('01/06/2020', '%d/%m/%Y')
end = datetime.strptime('18/06/2020', '%d/%m/%Y')
cod_recinto = 52
lista_fma = get_lista_fma(cod_recinto, start, end)

In [10]:

lista_fma

[{'Sequencial': 1,
  'DT_DTA': 'DT',
  'Numero_FMA': 34,
  'Ano_FMA': 2020,
  'Data_Emissao': '2020-06-18',
  'CPF_Responsavel_Emissao': '35126903861',
  'Cod_Recinto': 52,
  'Nome_Recinto': 'EUDMARCO - PORTO SECO',
  'Cod_Operador': 843,
  'Nome_Operador': 'EMBRAPORT OPERADOR',
  'Data_Entrada_Navio': '2020-01-25 04:37:19',
  'Cod_Navio': 9723265,
  'Nome_Navio': 'SWITZERLAND',
  'Cod_Porto': 979,
  'Nome_Porto': 'YANTIAN',
  'Nacionalidade': 'CHINA',
  'Consignatario': 'AEREOMAR IMPORTADORA E EXPORTADORA LTDA',
  'Numero_Viagem_DTA': 151,
  'Ano_Viagem_DTA': 2020,
  'CE_Mercante': '152005007740364',
  'BL': 'WYTN9120447',
  'Lista_Containers': [{'Container': 'GCXU5104037'}],
  'Bagagem': 'NÃO',
  'Mercadoria': 'ARTIGOS DE TRANSPORTE OU',
  'Embalagem': 'VOLUME',
  'Quantidade': 653,
  'Peso': '27635',
  'Unidade_Peso': 'Kg',
  'Marca_Contramarca': '',
  'Obs': ''},
 {'Sequencial': 2,
  'DT_DTA': 'DT',
  'Numero_FMA': 32,
  'Ano_FMA': 2020,
  'Data_Emissao': '2020-06-18',
  'CPF_Respo

In [14]:
fmas = [(fma['Numero_FMA'], fma['Data_Emissao']) for fma in lista_fma]
print(sorted(fmas, key=lambda x: x[0]))

[(17, '2020-06-17'), (18, '2020-06-17'), (19, '2020-06-17'), (20, '2020-06-17'), (21, '2020-06-03'), (23, '2020-06-17'), (31, '2020-06-18'), (32, '2020-06-18'), (33, '2020-06-18'), (34, '2020-06-18'), (35, '2020-06-18'), (36, '2020-06-18'), (37, '2020-06-18'), (38, '2020-06-18'), (39, '2020-06-18'), (40, '2020-06-18'), (41, '2020-06-18'), (42, '2020-06-18'), (43, '2020-06-18'), (44, '2020-06-18'), (45, '2020-06-18'), (46, '2020-06-18'), (47, '2020-06-18'), (48, '2020-06-18'), (49, '2020-06-18'), (50, '2020-06-18'), (51, '2020-06-18'), (52, '2020-06-18'), (53, '2020-06-18')]


In [11]:
print(recintos_list[:5])
for linha in recintos_list[:5]:
    print(len(linha))

[['\ufeffCODIGO', 'CODE', 'NOME'], ['25', '025', 'BANDEIRANTES DE ARMAZENS GERAIS'], ['105', '105', 'BRACENTER CENTRO BRAS. DE ARMAZENAGEM E DISTRIBUICAO LTDA'], ['22', '022', 'Brasil Terminal Portuario'], ['846', '846', 'Brasil Terminal Portuario', 'Operador']]
3
3
3
3
4


In [14]:
for linha in recintos_list[1:]:
    recinto_id = int(linha[0])
    nome = linha[2]
    if len(linha) > 3:
        nome += ' - '  + linha[3]
    recinto = session.query(Recinto).filter(Recinto.id == recinto_id).one_or_none()
    if recinto is None:
        recinto = Recinto()
        print('Não achou %s' % recinto_id)
    recinto.id = recinto_id
    recinto.nome = nome
    try:
        session.add(recinto)
        session.commit()
        print(recinto.id, recinto.nome)
    except:
        session.rollback()
        
    


25 BANDEIRANTES DE ARMAZENS GERAIS
105 BRACENTER CENTRO BRAS. DE ARMAZENAGEM E DISTRIBUIC
22 Brasil Terminal Portuario
846 Brasil Terminal Portuario - Operador
24 BUNGE ALIMENTOS -MOINHO SANTOS
839 BUNGE ALIMENTOS-MOINHO SANTOS-OPERADOR PORTUARIO
45 CBA- CIA BRASILEIRA DE ALUMINIO
108 CIA BANDEIRANTES - REDEX CUBATÃO
103 CIA BANDEIRANTES - REDEX MANOEL TOURINHO
102 CIA BANDEIRANTES - REDEX UNIDADE CAUIBI
822 CIA. BANDEIRANTES - OPERADOR PORTUARIO
101 COLUMBIA - REDEX
12 DEICMAR - PAV
109 DEICMAR - REDEX SANTOS-BK
94 DEICMAR - TERMINAL MARITIMO DO VALONGO
96 DEICMAR ALEMOA - PORTO SECO
844 DEICMAR PORT S.A. - GUARUJÁ
837 DEICMAR PORT S.A. - SANTOS
973 DEICMAR S/A - OPERADOR PORTUARIO
104 DEICMAR S/A - REDEX
110 DEICMAR S/A - REDEX BORIS KAUFFMANN
97 DEICMAR SABOO - IPA
959 ECOPORTO SANTOS - OP. TECONDI
10 ECOPORTO SANTOS - TERMINAL 1
11 ECOPORTO SANTOS - TERMINAL 2
9 ECOPORTO SANTOS - TERMINAL 3
8 ECOPORTO SANTOS - TERMINAL 4
21 EMBRAPORT - EMPRESA BRASILEIRA DE TERMINAIS PORTUA
843 EMB

In [ ]:
qry = session.query(func.max(OVR.datahora).label("last_date"))
res = qry.execute()
start = res.last_date
end = datetime.today()
print(start, end)

In [15]:


for lista in recintos_list[1:]:
    cod_recinto = int(lista[0])
    lista_fma = get_lista_fma(cod_recinto, start, end)
    if lista_fma:
        for fma in lista_fma:
            ovr = session.query(OVR).filter(
                OVR.numero==fma['Numero_FMA'] & ovr.recinto_id == int(fma['Cod_Recinto'])
            ).one_or_none()
            if ovr:
                continue
            ovr = OVR()
            ovr.numero = fma['Numero_FMA']
            ovr.ano = fma['Ano_FMA']
            ovr.datahora = datetime.strptime(fma['Data_Emissao'], '%Y-%m-%d')
            ovr.recinto_id = int(fma['Cod_Recinto'])
            ovr.numeroCEmercante = fma['CE_Mercante']
            ovr.tipooperacao = 0
            ovr.fase = 0
            ovr.tipoevento_id = 1
            try:
                session.add(ovr)
                session.commit()
            except Exception as err:
                print(err)
                session.rollback()
     
                                     
# 'CPF_Responsavel_Emissao': '19934152894',
#    'Cod_Operador': 846,
#     'Data_Entrada_Navio': '2019-11-05 11:31:45',
#     'Cod_Navio': 9705005,
#     'Nome_Navio': 'MSC CATERINA',
#     'Cod_Porto': 14,
#     'Nome_Porto': 'ANTUERPIA (AMBERES)',
#     'Nacionalidade': 'BELGICA',
#     'Consignatario': 'BRASIL TERMINAL PORTUARIO S.A.',



C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpoo

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpoo

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpoo